In [1]:
from sklearn.linear_model import LogisticRegression
import numpy as np
# model = LogisticRegression()
# model.fit(가울기, 절편)

# 1. 데이터 불러오기

In [2]:
import pandas as pd
passengers = pd.read_csv('titanic_train.csv')
print(passengers.shape)
print(passengers.head())
print(passengers.columns)

(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN    

In [3]:
passengers.shape
#클래스 레이블 1이 살아남은거다.

(891, 12)

# 2. 데이터 전처리

## 1. 문자열(비정형)-> 숫자로 변환(정형)

In [4]:
passengers['Sex'] = passengers['Sex'].map({'female':1,'male':0})

In [5]:
passengers['Age'].fillna(value=passengers['Age'].mean(), inplace=True)

## 3. feature 분리하기
## sex,age,Pclass를 추출. 가설 : 여성>어린이>1/2/3등석 순으로 살아남았을 거다.

In [6]:
passengers['FirstClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 1 else 0)
passengers['SecondClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 2 else 0)

In [7]:
features = passengers[['Sex', 'Age', 'FirstClass', 'SecondClass']]
survival = passengers['Survived']

In [8]:
type(features.Sex)

pandas.core.series.Series

# 3. 학습세트/평가세트 분리

In [9]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, survival,test_size=0.3,random_state=1)

# 4. 데이터 정규화(scaleling)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

# 5. 모델 생성 및 평가하기

In [11]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_features,train_labels)

LogisticRegression()

In [12]:
model.score(train_features,train_labels)

0.8073836276083467

In [13]:
model.score(test_features,test_labels)

0.7686567164179104

In [14]:
#feature들의
model.coef_

array([[ 1.30129626, -0.46898282,  0.99138013,  0.55929388]])

# 6. 예측하기

In [17]:
Jack = np.array([0.0, 20.0, 0.0, 0.0])
Rose = np.array([1.0, 17.0, 1.0, 0.0])
ME = np.array([0.0,32.0,1.0,0.0])
sample_passengers = np.array([Jack,Rose, ME])

#훈련 데이터의 평균과 표준편차로 정규화
sample_passengers = scaler.transform(sample_passengers)

print(model.predict(sample_passengers))

[0 1 0]


In [18]:
model.predict_proba(sample_passengers)

array([[0.90194787, 0.09805213],
       [0.04879887, 0.95120113],
       [0.57240835, 0.42759165]])

In [19]:
model.predict_proba(sample_passengers).argmax(axis=1)

array([0, 1, 0], dtype=int64)